In [1]:
import sys
from functools import partial
sys.path.append("../")
import genjax
import numpy as np
import bayes3d as b
import jax.numpy as jnp
import bayes3d.genjax
import jax
from utils import *
from viz import *
from models import *
from renderer_setup import *
from genjax.inference.importance_sampling import sampling_importance_resampling
from genjax._src.core.transforms.incremental import NoChange
from genjax._src.core.transforms.incremental import UnknownChange
from genjax._src.core.transforms.incremental import Diff

console = genjax.pretty()

In [2]:
gt_path = "../ground_truths/genjax_generated/scene_6.pkl"
metadata = load_metadata(gt_path)
gt_images = metadata["rendered"]
model = eval("model_v{}".format(metadata["model_version"]))
RENDERER_ARGS = metadata["RENDERER_ARGS"]
setup_renderer_and_meshes = eval("setup_renderer_and_meshes_v{}".format(metadata["renderer_setup_version"]))
setup_renderer_and_meshes(**RENDERER_ARGS)

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (64, 64, 1024)


In [3]:
def c2f_pose_update_v2(trace_, key, pose_grid, enumerator):
    
    scores = enumerator.enumerate_choices_get_scores(trace_, key, pose_grid)
    return enumerator.update_choices(
        trace_, key,
        pose_grid[scores.argmax()]
    )
c2f_pose_update_v2_jit = jax.jit(c2f_pose_update_v2, static_argnames=("enumerator",))

In [25]:
def inference_approach_F(model, gt, metadata):
    """
    2-step model with NO unfold
    HMM-style
    """
    # Use 3d translation and rotation grid
    grid_widths = [0.2,0.1,0.05, 0.025, 0.0125]
    grid_nums = [(3,3,3),(3,3,3),(3,3,3),(3,3,3),(3,3,3)]
    gridding_schedule = make_schedule_3d(grid_widths,grid_nums, [-jnp.pi/12, jnp.pi/12],10,10,jnp.pi/40)

    key = jax.random.PRNGKey(metadata["key_number"]+71)
    base_chm = genjax.choice_map(metadata["CHOICE_MAP_ARGS"])
    enumerator = b.make_enumerator(["velocity"])
    pose = metadata["INIT_POSE"]
    velocity = metadata["INIT_VELOCITY"]
    T = metadata["T"]
    traces = []
    model_args = metadata["MODEL_ARGS"]

    for t in range(1,T+1):
        print("t = ", t)
        # force new constaints values to take over
        chm = base_chm.unsafe_merge(genjax.choice_map({
            "depth" : gt[t]
        }))

        model_args["pose"] = pose
        model_args["velocity"] = velocity
        # RESORTING to model.importance as I am having issues with update and choicemaps with unfolds &/or maps
        _, trace = model.importance(key, chm, tuple(model_args.values()))

        # then update trace over all the proposals
        for i, grid in enumerate(gridding_schedule):
            # print("Grid #",i+1)
            trace = c2f_pose_update_v2_jit(trace, key, grid, enumerator)
        pose, velocity = trace.get_retval()[1]
        traces.append(trace)

    # first gt image can be assumed to be known as we have the init pose
    rendered = jnp.stack([gt[0]]+[tr.get_retval()[0] for tr in traces])
    return traces, rendered

In [26]:
trs, rendered = inference_approach_F(model, gt_images, metadata)

t =  1
t =  2
t =  3
t =  4
t =  5


In [27]:
print(np.sum(metadata["scores"]))
print(np.sum([tr.get_score() for tr in trs]))

40454.742
32882.62


In [28]:
# video_from_trace(tr, framerate=5, use_retval=True)
# video_from_rendered(gt_images, framerate=5,)
# video_comparison_from_trace(tr,framerate = 5, scale = 4)
video_comparison_from_images(rendered, gt_images,framerate = 5, scale = 4)

<IPython.core.display.HTML object>

In [20]:
chm[2]